In [1]:
import pandas as pd

# Question 1

##### a)  Imputation Using (Most Frequent) or (Zero/Constant) Values or Imputation Using Deep Learning (Datawig)

##### b) sensitive to outliers = Imputation Using k-NN
#####     considers all columns values = Imputation Using k-NN

##### c) Pros: Works well with categorical features.
#####     Cons: It also doesn’t factor the correlations between features. It can introduce bias in the data.

# Question 2

In [2]:
nba = pd.read_csv("nba.csv")

##### a)

In [3]:
#creates sample for use in following questions, includes 80% of nba data, random state included for stability
nbasample = nba.sample(frac = .8, random_state = 2)

In [4]:
#gets original info for memory usage in question h
nbasample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366 entries, 117 to 409
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      366 non-null    object 
 1   Team      366 non-null    object 
 2   Number    366 non-null    float64
 3   Position  366 non-null    object 
 4   Age       366 non-null    float64
 5   Height    366 non-null    object 
 6   Weight    366 non-null    float64
 7   College   301 non-null    object 
 8   Salary    355 non-null    float64
dtypes: float64(4), object(5)
memory usage: 28.6+ KB


##### b)

In [5]:
#number of missing values in each column - only 65 in college and 11 in salary
nbasample.isnull().sum()

Name         0
Team         0
Number       0
Position     0
Age          0
Height       0
Weight       0
College     65
Salary      11
dtype: int64

##### c)

In [6]:
#keeps only columns that have any value not equal to zero 
#(equiv of deleting all columns that only have zeroes) - doesn't do anything in this case
nbasample = nbasample.loc[(nbasample!=0).any(axis=1)]

##### d)

In [7]:
#fills college column with mode value
modecollege = nbasample.College.mode()
nbasample.College.fillna(modecollege[0], inplace = True)

In [8]:
#fills salary column with mean value
nbasample.Salary.fillna(nbasample.Salary.mean(), inplace = True)

##### e)

In [9]:
#set number and salary columns to int
nbasample.Number = nbasample.Number.astype('int')
nbasample.Salary = nbasample.Salary.astype('int')

##### f)

In [10]:
#display unique positions in Position column - C, PF, SG, SF, PG
nbasample.Position.unique()

array(['C', 'PF', 'SG', 'SF', 'PG'], dtype=object)

In [11]:
#convert position column to category - only 5 values out of 366 data points
nbasample.Position.astype('category')

117     C
281    PF
372    PF
20     SG
221    SF
       ..
123    SF
311    SG
16     SG
224     C
409    PF
Name: Position, Length: 366, dtype: category
Categories (5, object): [C, PF, PG, SF, SG]

##### g)

In [12]:
#30 unique teams in nbasample - 366 data points
nbasample.Team.nunique()

30

In [13]:
#unique values less than 10 times the total number of data points in Team columns - data type to be changes
if nbasample.Team.nunique() * 10 < nbasample.Team.count():
    print('less than 10x')
else:
    print('more than 10x')

less than 10x


In [36]:
#changes dtype to category since nunique is less than 10x the number of data points
nbasample.Team = nbasample.Team.astype("category")

##### h)

In [37]:
#memory usage goes from 28.6+ to 27.6+, and has decreased.
nbasample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366 entries, 117 to 409
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Name      366 non-null    object  
 1   Team      366 non-null    category
 2   Number    366 non-null    int32   
 3   Position  366 non-null    object  
 4   Age       366 non-null    float64 
 5   Height    366 non-null    object  
 6   Weight    366 non-null    float64 
 7   College   366 non-null    object  
 8   Salary    366 non-null    int32   
 9   agegroup  366 non-null    object  
dtypes: category(1), float64(2), int32(2), object(5)
memory usage: 27.6+ KB


##### i)

In [39]:
#creates new sorted data set with Team sorted by asccending, salary by descending, and resets index
nbasamplesorted = nbasample.sort_values(["Team", "Salary"], ascending = (True, False)).reset_index()
nbasamplesorted

,index,Name,Team,Number,Position,Age,Height,Weight,College,Salary,agegroup
0,315,Paul Millsap,Atlanta Hawks,4,PF,31.0,6-8,246.0,Louisiana Tech,18671659,middle
1,312,Al Horford,Atlanta Hawks,15,C,30.0,6-10,245.0,Florida,12000000,middle
2,321,Tiago Splitter,Atlanta Hawks,11,C,31.0,6-11,245.0,Kentucky,9756250,middle
3,314,Kyle Korver,Atlanta Hawks,26,SG,35.0,6-7,212.0,Creighton,5746479,middle
4,320,Thabo Sefolosha,Atlanta Hawks,25,SF,32.0,6-7,220.0,Kentucky,4000000,middle
...,...,...,...,...,...,...,...,...,...,...,...
361,379,Ramon Sessions,Washington Wizards,7,PG,30.0,6-3,190.0,Nevada,2170465,middle
362,377,Kelly Oubre Jr.,Washington Wizards,12,SF,20.0,6-7,205.0,Kansas,1920240,young
363,371,Jarell Eddie,Washington Wizards,8,SG,24.0,6-7,218.0,Virginia Tech,561716,young
364,374,JJ Hickson,Washington Wizards,21,C,27.0,6-9,242.0,North Carolina State,273038,middle


##### j)

In [16]:
#creates a column to group for different age groups
nbasample["agegroup"] = nbasample.Age.apply(lambda x : 
                 "old" if x >35 else 
                 "young" if x < 25 else 
                 "middle")

In [17]:
#middle age between 25 and 35 receives the highest average salary
nbasample.groupby(by = "agegroup").mean()

,Number,Age,Weight,Salary
agegroup,,,,
middle,17.838565,28.511211,222.726457,5.974205e+06
old,24.125000,37.562500,225.187500,4.810857e+06
young,17.433071,22.472441,218.913386,2.783634e+06


# Question 3

In [18]:
foods = pd.read_csv("foods.csv")

In [19]:
foods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   First Name  1000 non-null   object 
 1   Gender      1000 non-null   object 
 2   City        1000 non-null   object 
 3   Frequency   1000 non-null   object 
 4   Item        1000 non-null   object 
 5   Spend       1000 non-null   float64
dtypes: float64(1), object(5)
memory usage: 47.0+ KB


In [20]:
foods.head()

,First Name,Gender,City,Frequency,Item,Spend
0,Wanda,Female,Stamford,Weekly,Burger,15.66
1,Eric,Male,Stamford,Daily,Chalupa,10.56
2,Charles,Male,New York,Never,Sushi,42.14
3,Anna,Female,Philadelphia,Once,Ice Cream,11.01
4,Deborah,Female,Philadelphia,Daily,Chalupa,23.49


##### a)

In [21]:
#change all but first name to category - first name has a large number unique values
foods = foods.astype({'Gender' : 'category','City' : 'category','Frequency' : 'category','Item' : 'category'})

In [22]:
foods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   First Name  1000 non-null   object  
 1   Gender      1000 non-null   category
 2   City        1000 non-null   category
 3   Frequency   1000 non-null   category
 4   Item        1000 non-null   category
 5   Spend       1000 non-null   float64 
dtypes: category(4), float64(1), object(1)
memory usage: 20.4+ KB


##### b)

In [23]:
#spend average by gender using groupby
foods.groupby(by = "Gender").mean()

,Spend
Gender,
Female,50.709629
Male,49.397623


In [24]:
#spend average by gender using pivot_table
foods.pivot_table(values = "Spend", index = "Gender", aggfunc = "mean")

,Spend
Gender,
Female,50.709629
Male,49.397623


##### c)

In [25]:
#sum of money spent by each gender
foods.groupby(by = "Gender").sum()

,Spend
Gender,
Female,25963.33
Male,24106.04


##### d)

In [26]:
#spend average by gender, item, and city using groupby
foods.groupby(by = ["Gender", "Item", "City"]).mean()

Spend
Gender Item      City                   
Female Burger    New York      51.626667
                 Philadelphia  52.878710
                 Stamford      45.037778
       Burrito   New York      42.563043
                 Philadelphia  52.098571
                 Stamford      53.532647
       Chalupa   New York      46.135789
                 Philadelphia  52.291563
                 Stamford      64.094000
       Donut     New York      46.670323
                 Philadelphia  54.642000
                 Stamford      48.734118
       Ice Cream New York      56.356296
                 Philadelphia  46.225625
                 Stamford      46.910455
       Sushi     New York      47.751290
                 Philadelphia  58.096000
                 Stamford      45.622187
Male   Burger    New York      58.822273
                 Philadelphia  44.675238
                 Stamford      46.424516
       Burrito   New York      55.976000
                 Philadelphia  43.764333
                 Stamford      46.438929
       Chalupa   New York      49.110800
                 Philadelphia  48.444783
                 Stamford      50.011304
       Donut     New York      44.842333
                 Philadelphia  37.859394
                 Stamford      49.004483
       Ice Cream New York      55.297586
                 Philadelphia  53.445610
                 Stamford      42.368800
       Sushi     New York      51.709259
                 Philadelphia  49.852857
                 Stamford      70.434444

In [27]:
#spend average by gender, item, and city using pivot_table
foods.pivot_table(values = "Spend", index = ["Gender", "Item", "City"], aggfunc = "mean")

Spend
Gender Item      City                   
Female Burger    New York      51.626667
                 Philadelphia  52.878710
                 Stamford      45.037778
       Burrito   New York      42.563043
                 Philadelphia  52.098571
                 Stamford      53.532647
       Chalupa   New York      46.135789
                 Philadelphia  52.291563
                 Stamford      64.094000
       Donut     New York      46.670323
                 Philadelphia  54.642000
                 Stamford      48.734118
       Ice Cream New York      56.356296
                 Philadelphia  46.225625
                 Stamford      46.910455
       Sushi     New York      47.751290
                 Philadelphia  58.096000
                 Stamford      45.622187
Male   Burger    New York      58.822273
                 Philadelphia  44.675238
                 Stamford      46.424516
       Burrito   New York      55.976000
                 Philadelphia  43.764333
                 Stamford      46.438929
       Chalupa   New York      49.110800
                 Philadelphia  48.444783
                 Stamford      50.011304
       Donut     New York      44.842333
                 Philadelphia  37.859394
                 Stamford      49.004483
       Ice Cream New York      55.297586
                 Philadelphia  53.445610
                 Stamford      42.368800
       Sushi     New York      51.709259
                 Philadelphia  49.852857
                 Stamford      70.434444

# Question 4

In [28]:
week1sales = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2sales = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv")
foods = pd.read_csv("Restaurant - Foods.csv")

##### a)

In [29]:
#creates df where only unique customer values remain, with no duplicates
didnotreturn = pd.concat((week1sales, week2sales)).drop_duplicates("Customer ID")

In [30]:
didnotreturn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399 entries, 0 to 248
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Customer ID  399 non-null    int64
 1   Food ID      399 non-null    int64
dtypes: int64(2)
memory usage: 9.4 KB


In [31]:
#399 customers ordered food in either week 1 or week 2 but not both
didnotreturn.nunique()

Customer ID    399
Food ID         10
dtype: int64

##### b)

In [32]:
#merges week 2 sales data and customers data, dropping ID to avoid duplicate key columns
week2customers = pd.merge(week2sales, customers, how = 'inner', left_on = "Customer ID", right_on = "ID").drop("ID", axis = 1)

In [35]:
#merges previous frame with foods frame, to get full comparison
week2foodcustomers = pd.merge(week2customers, foods, how = 'left', on = "Food ID")

In [34]:
#fully merged data set with customers, sales, and food data
#(question is ambiguous, it mentions both customers, week 2, and food, so I created a fully merged data set)
#(states to consider week 2 sales and customers, but then says to merge so it shows food info with week 2 customers)
week2foodcustomers

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,688,10,Carl,Williamson,Male,Thoughtmix,Graphic Designer,Drink,1.75
1,813,7,Johnny,Walker,Male,Kayveo,Developer II,Steak,24.99
2,495,10,Deborah,Little,Female,Babbleblab,VP Accounting,Drink,1.75
3,495,6,Deborah,Little,Female,Babbleblab,VP Accounting,Pasta,13.99
4,495,2,Deborah,Little,Female,Babbleblab,VP Accounting,Burrito,9.99
...,...,...,...,...,...,...,...,...,...
245,945,4,Anne,Reed,Female,Twitterworks,Director of Sales,Quesadilla,4.25
246,783,10,Phyllis,Meyer,Female,Voolia,Information Systems Manager,Drink,1.75
247,556,10,Samuel,Bailey,Male,Oyoloo,Nurse,Drink,1.75
248,252,9,Douglas,Powell,Male,Jetwire,Geologist IV,Donut,0.99
